In [21]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

## Load Model

In [22]:
filename_pickle = 'model/final_model.pkl'

In [23]:
with open(filename_pickle, 'rb') as handle:
    model = pickle.load(handle)

## User Input

In [24]:
DATE = '06/26/17'
CRS_DEP_TIME = 1103
UNIQUE_CARRIER = 'UA'
FL_NUM = 292
ORIGIN_CITY_NAME = 'San Francisco, CA'
DEST_CITY_NAME = 'Orlando, FL'

In [25]:
DepHr = CRS_DEP_TIME//100
date_obj = datetime.strptime(DATE,'%m/%d/%y')
MONTH = date_obj.month
DAY_OF_WEEK = date_obj.weekday()+1 #DAY_OF_WEEK in original data 1-7

In [26]:
input_var = ('MONTH','DAY_OF_WEEK','DepHr','UNIQUE_CARRIER','FL_NUM','ORIGIN_CITY_NAME','DEST_CITY_NAME')
input_dict = {}
for i in input_var:
    input_dict[i] = locals()[i]
input_dict

{'DAY_OF_WEEK': 1,
 'DEST_CITY_NAME': 'Orlando, FL',
 'DepHr': 11,
 'FL_NUM': 292,
 'MONTH': 6,
 'ORIGIN_CITY_NAME': 'San Francisco, CA',
 'UNIQUE_CARRIER': 'UA'}

## Add Metadata (ORIGIN/DEST/DISTANCE)

In [27]:
metadata = pd.read_csv('data/metadata.csv')

In [28]:
df_lookup = pd.DataFrame(input_dict, index=['i',])

In [29]:
on_cols = ['UNIQUE_CARRIER','FL_NUM','ORIGIN_CITY_NAME','DEST_CITY_NAME']
df_input = pd.merge(df_lookup, metadata, how='left', on=on_cols)

In [34]:
if df_input.isnull().values.any():
    print 'Historic metadata (ORIGIN/DEST/DISTANCE) needed for modeling not found!'
df_input

,DAY_OF_WEEK,DEST_CITY_NAME,DepHr,FL_NUM,MONTH,ORIGIN_CITY_NAME,UNIQUE_CARRIER,ORIGIN,DEST,DISTANCE
0,1,"Orlando, FL",11,292,6,"San Francisco, CA",UA,SFO,MCO,2446.0


## Prediction

In [37]:
print (model.classes_)
predictions = model.predict_proba(df_input)
print (predictions)

[ 0.  1.]
[[ 0.78965513  0.21034487]]


/usr/local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [44]:
print ('The probability of delay is {:.0%}'.format(predictions[0][1]))

The probability of delay is 21%
